In [1]:
import sys

sys.path

sys.path.append("/Users/hbuhrmann/PycharmProjects/CycleStats")

In [3]:
import sourcetocsv as cycle
import numpy as np
import thinkstats2
import thinkplot
import pandas as pd

In [4]:
#filepath = 'C:/Users/hanne/PycharmProjects/CycleStats/PowerOutputExperiments/FinalFiles/'
filepath = '/users/hbuhrmann/PycharmProjects/CycleStats/PowerOutputExperiments/FinalFiles/'
df = cycle.getfinalcsvcycledata(filepath,'_last_7_rides.csv')

df['CumEnergy']=df.Watts.cumsum()
df['TenSecPowerMean'] = df.Watts.rolling(window=10).mean()
df['MinutePowerMean'] = df.Watts.rolling(window=60).mean()
df['FiveMinutePowerMean'] = df.Watts.rolling(window=300).mean()

df2 = df[df.IncludeIn2 == 1].copy(deep=True)


In [ ]:
# PREPARATION WORK FOR PROPER FANCY CALCULATED POWER, FROM THIS LINK :
# https://www.gribble.org/cycling/power_v_speed.html

Gradient = 9
#74.134 4.102 6.068 11.11 0.09 2000.0 468.39
#73.786 4.102 19.846 20.1 0.09 2000.0 982.013
WeightPerson = 75   #kg
WeightBike = 9 #kg
Weight = WeightPerson+WeightBike

g = 9.8067 #Gravity
Crr = 0.005  #Coefficient of rolling resistance
Rho = 1.22601    #Air density in kg/m3
Cd = 0.63   #Coefficient of drag 
Fa = 0.509  # Frontal area in sq m

#CycleSpeed = 20  #km/h
WindSpeed = 0 #km/h
WindDir = np.pi

CycleDistance = 20.11  #km
TravelDir = np.pi

Duration = 2000   #milliseconds

FGravity = g * np.sin(np.arctan(Gradient/100))*Weight

FRolling = g * np.cos(np.arctan(Gradient/100))*Weight*Crr


#This formula does not yet take windspeed and wind direction and direction of travel into account
FDrag = 0.5*Cd*Fa*Rho*(CycleDistance/(Duration/1000))**2

FResist = FGravity+FRolling+FDrag

print(f' FGravity = {FGravity}, FRolling = {FRolling}, FDrag = {FDrag}, FResist = {FResist}')

Work = FResist * CycleDistance

Power = Work / (Duration / 1000)

print(f'Work = {Work}, Power = {Power}')

EKinetic = 0.5*Weight*(CycleDistance/Duration*1000)**2  #1/2mv**2

print (f'Kinetic Energy = {EKinetic}')


In [5]:
Gradient = 0
WeightPerson = 75   #kg
WeightBike = 9 #kg
Weight = WeightPerson+WeightBike

g = 9.8067 #Gravity
Crr = 0.0025  #Coefficient of rolling resistance
Rho = 1.22601    #Air density in kg/m3
Cd = 0.63   #Coefficient of drag 
Fa = 0.509  # Frontal area in sq m

WindSpeed = 10 #km/h
WindDir = np.pi
TravelDir = np.pi

df2['FGravity'] = g * np.sin(np.arctan(df2.Gradient2))*Weight*df2.Elevation2

df2['FRolling'] = g * np.cos(np.arctan(df2.Gradient2))*Weight*Crr*df2.DistXYZ2

#This formula does not yet take windspeed and wind direction and direction of travel into account
df2['FDrag'] = 0.5*Cd*Fa*Rho*(df2.DistXYZ2/(df2.Duration2/1000))**2*(df2.Duration2/1000)

df2['FAccel'] = (df2.Speedmps2 - df2.Speedmps2.shift(periods=1))*Weight

df2['FResist'] = df2.FGravity+df2.FRolling+df2.FDrag


#df2['CalculatedWork'] = df2.FResist * df2.DistXYZ2 + df2.FAccel

#df2['CalculatedPower'] = df2.CalculatedWork / (df2.Duration2 / 1000)


df2['CalculatedWork'] = (df2.FResist + df2.FAccel)*(df.Duration2 / 1000)

df2['CalculatedPower'] = df2.CalculatedWork / (df2.Duration2 / 1000)



def disp3 (n):
    if np.isnan(n) == True:
        return "     "
    return '{:05.2f}'.format(n)
    #return round(n,3)

for index,row in df2.iterrows():
    
    output = 'Dist:'+disp3(row.DistXYZ2)
    output += ', Grad:'+disp3(row.Gradient2)
    output += ', Grav:'+disp3(row.FGravity)
    output += ', Roll:'+disp3(row.FRolling)
    output += ', Drag:'+disp3(row.FDrag)
    output += ', Resist:'+disp3(row.FResist)
    output += ', Accel:'+disp3(row.FAccel)
    output += ', Speed:'+disp3(row.Speedmps2)
    output += ', Work:'+disp3(row.CalculatedWork)
    output += ', Power:' +disp3(row.CalculatedPower)

    print(output)



Dist:     , Grad:     , Grav:     , Roll:     , Drag:     , Resist:     , Accel:     , Speed:     , Work:     , Power:     
Dist:00.00, Grad:     , Grav:     , Roll:     , Drag:00.00, Resist:     , Accel:     , Speed:00.00, Work:     , Power:     
Dist:09.54, Grad:00.00, Grav:00.00, Roll:19.65, Drag:08.95, Resist:28.60, Accel:400.82, Speed:04.77, Work:858.84, Power:429.42
Dist:09.34, Grad:00.00, Grav:00.00, Roll:19.23, Drag:08.57, Resist:27.79, Accel:-8.72, Speed:04.67, Work:38.14, Power:19.07
Dist:10.04, Grad:00.00, Grav:00.00, Roll:20.68, Drag:09.91, Resist:30.59, Accel:29.65, Speed:05.02, Work:120.49, Power:60.24
Dist:10.38, Grad:00.00, Grav:00.00, Roll:21.38, Drag:10.59, Resist:31.97, Accel:14.25, Speed:05.19, Work:92.45, Power:46.22
Dist:21.74, Grad:00.00, Grav:00.00, Roll:44.77, Drag:46.46, Resist:91.23, Accel:477.13, Speed:10.87, Work:1136.73, Power:568.37
Dist:11.71, Grad:-0.05, Grav:25.32, Roll:24.09, Drag:13.49, Resist:62.90, Accel:-421.13, Speed:05.86, Work:-716.47, Power:-3

Dist:15.98, Grad:00.00, Grav:00.00, Roll:32.91, Drag:25.10, Resist:58.01, Accel:-45.25, Speed:07.99, Work:25.51, Power:12.76
Dist:15.42, Grad:00.00, Grav:00.00, Roll:31.76, Drag:23.37, Resist:55.13, Accel:-23.48, Speed:07.71, Work:63.29, Power:31.65
Dist:15.32, Grad:00.07, Grav:53.76, Roll:31.49, Drag:23.08, Resist:108.33, Accel:-4.07, Speed:07.66, Work:208.53, Power:104.26
Dist:14.20, Grad:00.06, Grav:37.13, Roll:29.19, Drag:19.81, Resist:86.14, Accel:-47.30, Speed:07.10, Work:77.68, Power:38.84
Dist:21.69, Grad:00.02, Grav:06.08, Roll:44.66, Drag:46.25, Resist:96.99, Accel:314.76, Speed:10.85, Work:823.49, Power:411.75
Dist:13.63, Grad:00.00, Grav:00.00, Roll:28.06, Drag:18.25, Resist:46.31, Accel:-338.72, Speed:06.81, Work:-584.81, Power:-292.41
Dist:16.37, Grad:00.02, Grav:08.06, Roll:33.69, Drag:26.32, Resist:68.07, Accel:115.00, Speed:08.18, Work:366.15, Power:183.07
Dist:14.51, Grad:00.07, Grav:56.77, Roll:29.82, Drag:20.70, Resist:107.28, Accel:-77.81, Speed:07.26, Work:58.95, 

Dist:26.80, Grad:00.00, Grav:00.00, Roll:55.20, Drag:70.60, Resist:125.80, Accel:305.88, Speed:13.40, Work:863.35, Power:431.67
Dist:21.91, Grad:00.00, Grav:00.00, Roll:45.12, Drag:47.18, Resist:92.30, Accel:-205.49, Speed:10.95, Work:-226.37, Power:-113.19
Dist:23.18, Grad:00.00, Grav:00.00, Roll:47.73, Drag:52.80, Resist:100.53, Accel:53.29, Speed:11.59, Work:307.66, Power:153.83
Dist:27.97, Grad:00.00, Grav:00.00, Roll:57.60, Drag:76.88, Resist:134.47, Accel:201.16, Speed:13.98, Work:671.26, Power:335.63
Dist:17.42, Grad:00.00, Grav:00.00, Roll:35.87, Drag:29.82, Resist:65.69, Accel:-443.06, Speed:08.71, Work:-754.73, Power:-377.36
Dist:26.11, Grad:00.00, Grav:00.00, Roll:53.77, Drag:67.00, Resist:120.78, Accel:365.04, Speed:13.05, Work:971.63, Power:485.82
Dist:19.39, Grad:00.00, Grav:00.00, Roll:39.92, Drag:36.94, Resist:76.86, Accel:-282.43, Speed:09.69, Work:-411.15, Power:-205.57
Dist:31.17, Grad:00.00, Grav:00.00, Roll:64.19, Drag:95.48, Resist:159.67, Accel:494.86, Speed:15.5

Dist:15.55, Grad:00.13, Grav:211.97, Roll:31.75, Drag:23.75, Resist:267.47, Accel:257.06, Speed:07.77, Work:1049.07, Power:524.53
Dist:06.11, Grad:00.16, Grav:133.37, Roll:12.41, Drag:03.67, Resist:149.45, Accel:-396.35, Speed:03.05, Work:-493.80, Power:-246.90
Dist:10.74, Grad:00.09, Grav:76.71, Roll:22.03, Drag:11.34, Resist:110.08, Accel:194.60, Speed:05.37, Work:609.35, Power:304.68
Dist:15.31, Grad:00.09, Grav:105.46, Roll:31.40, Drag:23.04, Resist:159.90, Accel:191.88, Speed:07.66, Work:703.56, Power:351.78
Dist:10.22, Grad:00.10, Grav:80.60, Roll:20.95, Drag:10.27, Resist:111.82, Accel:-213.69, Speed:05.11, Work:-203.72, Power:-101.86
Dist:09.99, Grad:00.10, Grav:82.48, Roll:20.47, Drag:09.81, Resist:112.76, Accel:-9.76, Speed:04.99, Work:205.98, Power:102.99
Dist:15.24, Grad:00.11, Grav:138.41, Roll:31.20, Drag:22.82, Resist:192.43, Accel:220.36, Speed:07.62, Work:825.58, Power:412.79
Dist:10.25, Grad:00.04, Grav:12.87, Roll:21.09, Drag:10.32, Resist:44.28, Accel:-209.56, Speed

Dist:14.27, Grad:00.00, Grav:00.00, Roll:29.38, Drag:20.01, Resist:49.39, Accel:215.83, Speed:07.13, Work:530.44, Power:265.22
Dist:12.47, Grad:00.03, Grav:10.57, Roll:25.67, Drag:15.29, Resist:51.53, Accel:-75.46, Speed:06.24, Work:-47.86, Power:-23.93
Dist:07.68, Grad:00.13, Grav:107.37, Roll:15.67, Drag:05.79, Resist:128.84, Accel:-201.39, Speed:03.84, Work:-145.11, Power:-72.56
Dist:13.50, Grad:00.07, Grav:61.03, Roll:27.72, Drag:17.91, Resist:106.66, Accel:244.51, Speed:06.75, Work:702.34, Power:351.17
Dist:08.38, Grad:00.10, Grav:62.94, Roll:17.17, Drag:06.90, Resist:87.01, Accel:-215.11, Speed:04.19, Work:-256.20, Power:-128.10
Dist:11.99, Grad:00.12, Grav:134.61, Roll:24.53, Drag:14.14, Resist:173.28, Accel:151.97, Speed:06.00, Work:650.50, Power:325.25
Dist:07.04, Grad:00.00, Grav:00.00, Roll:14.49, Drag:04.87, Resist:19.36, Accel:-208.21, Speed:03.52, Work:-377.70, Power:-188.85
Dist:14.77, Grad:00.00, Grav:00.00, Roll:30.41, Drag:21.43, Resist:51.84, Accel:324.60, Speed:07.3

...

Dist:14.69, Grad:00.04, Grav:20.20, Roll:30.24, Drag:21.22, Resist:71.66, Accel:54.88, Speed:07.35, Work:253.07, Power:126.53
Dist:21.34, Grad:00.00, Grav:00.00, Roll:43.96, Drag:44.77, Resist:88.73, Accel:279.28, Speed:10.67, Work:736.02, Power:368.01
Dist:14.65, Grad:00.00, Grav:00.00, Roll:30.17, Drag:21.09, Resist:51.26, Accel:-281.18, Speed:07.32, Work:-459.83, Power:-229.92
Dist:14.40, Grad:00.00, Grav:00.00, Roll:29.65, Drag:20.37, Resist:50.02, Accel:-10.57, Speed:07.20, Work:78.90, Power:39.45
Dist:22.68, Grad:00.00, Grav:00.00, Roll:46.70, Drag:50.54, Resist:97.24, Accel:347.72, Speed:11.34, Work:889.92, Power:444.96
Dist:13.29, Grad:00.00, Grav:00.00, Roll:27.38, Drag:17.37, Resist:44.75, Accel:-394.06, Speed:06.65, Work:-698.63, Power:-349.31
Dist:19.45, Grad:00.00, Grav:00.00, Roll:40.06, Drag:37.18, Resist:77.24, Accel:258.57, Speed:09.73, Work:671.61, Power:335.81
Dist:13.72, Grad:00.00, Grav:00.00, Roll:28.26, Drag:18.50, Resist:46.76, Accel:-240.66, Speed:06.86, Work:-

Dist:08.73, Grad:00.05, Grav:15.12, Roll:17.95, Drag:07.48, Resist:40.55, Accel:33.61, Speed:04.36, Work:148.33, Power:74.17
Dist:12.99, Grad:00.00, Grav:00.00, Roll:26.76, Drag:16.59, Resist:43.35, Accel:179.24, Speed:06.50, Work:445.18, Power:222.59
Dist:07.89, Grad:00.00, Grav:00.00, Roll:16.24, Drag:06.11, Resist:22.35, Accel:-214.50, Speed:03.94, Work:-384.29, Power:-192.14
Dist:11.70, Grad:00.05, Grav:25.41, Roll:24.06, Drag:13.46, Resist:62.93, Accel:160.23, Speed:05.85, Work:446.32, Power:223.16
Dist:03.68, Grad:00.11, Grav:35.86, Roll:07.52, Drag:01.33, Resist:44.71, Accel:-337.07, Speed:01.84, Work:-584.71, Power:-292.36
Dist:15.44, Grad:00.05, Grav:34.16, Roll:31.75, Drag:23.42, Resist:89.32, Accel:493.93, Speed:07.72, Work:1166.50, Power:583.25
Dist:09.94, Grad:00.04, Grav:13.27, Roll:20.46, Drag:09.72, Resist:43.44, Accel:-230.69, Speed:04.97, Work:-374.49, Power:-187.24
Dist:09.40, Grad:00.00, Grav:00.00, Roll:19.36, Drag:08.69, Resist:28.04, Accel:-22.79, Speed:04.70, Wo

Dist:12.04, Grad:00.03, Grav:10.95, Roll:24.79, Drag:14.26, Resist:50.00, Accel:-6.86, Speed:06.02, Work:86.27, Power:43.14
Dist:05.90, Grad:00.00, Grav:00.00, Roll:12.16, Drag:03.43, Resist:15.59, Accel:-257.87, Speed:02.95, Work:-484.58, Power:-242.29
Dist:11.26, Grad:00.09, Grav:73.17, Roll:23.10, Drag:12.46, Resist:108.73, Accel:224.96, Speed:05.63, Work:667.38, Power:333.69
Dist:10.91, Grad:00.07, Grav:48.32, Roll:22.41, Drag:11.70, Resist:82.43, Accel:-14.73, Speed:05.45, Work:135.41, Power:67.70
Dist:10.54, Grad:00.04, Grav:12.52, Roll:21.68, Drag:10.91, Resist:45.11, Accel:-15.66, Speed:05.27, Work:58.92, Power:29.46
Dist:10.42, Grad:00.10, Grav:79.04, Roll:21.37, Drag:10.68, Resist:111.09, Accel:-4.76, Speed:05.21, Work:212.66, Power:106.33
Dist:10.34, Grad:00.10, Grav:79.66, Roll:21.20, Drag:10.51, Resist:111.37, Accel:-3.42, Speed:05.17, Work:215.91, Power:107.95
Dist:10.68, Grad:00.04, Grav:12.35, Roll:21.98, Drag:11.21, Resist:45.54, Accel:14.19, Speed:05.34, Work:119.45, 

Dist:12.02, Grad:00.07, Grav:43.88, Roll:24.69, Drag:14.19, Resist:82.76, Accel:128.97, Speed:06.01, Work:423.46, Power:211.73
Dist:09.95, Grad:00.00, Grav:00.00, Roll:20.49, Drag:09.73, Resist:30.21, Accel:-86.82, Speed:04.97, Work:-113.22, Power:-56.61
Dist:09.56, Grad:00.08, Grav:55.15, Roll:19.62, Drag:08.98, Resist:83.75, Accel:-16.33, Speed:04.78, Work:134.83, Power:67.42
Dist:09.28, Grad:00.11, Grav:88.77, Roll:19.00, Drag:08.47, Resist:116.24, Accel:-11.67, Speed:04.64, Work:209.13, Power:104.57
Dist:08.29, Grad:00.05, Grav:15.92, Roll:17.05, Drag:06.75, Resist:39.72, Accel:-41.67, Speed:04.14, Work:-3.89, Power:-1.95
Dist:11.27, Grad:00.07, Grav:46.79, Roll:23.15, Drag:12.48, Resist:82.41, Accel:125.11, Speed:05.63, Work:415.04, Power:207.52
Dist:09.76, Grad:00.10, Grav:84.44, Roll:19.99, Drag:09.36, Resist:113.79, Accel:-63.39, Speed:04.88, Work:100.79, Power:50.40
Dist:13.77, Grad:00.09, Grav:86.13, Roll:28.26, Drag:18.64, Resist:133.03, Accel:168.62, Speed:06.89, Work:603.2

Dist:11.44, Grad:00.00, Grav:00.00, Roll:23.56, Drag:12.87, Resist:36.43, Accel:-43.29, Speed:05.72, Work:-13.72, Power:-6.86
Dist:12.31, Grad:00.00, Grav:00.00, Roll:25.35, Drag:14.89, Resist:40.23, Accel:36.38, Speed:06.15, Work:153.22, Power:76.61
Dist:11.49, Grad:00.09, Grav:71.71, Roll:23.57, Drag:12.97, Resist:108.25, Accel:-34.40, Speed:05.74, Work:147.70, Power:73.85
Dist:06.22, Grad:00.06, Grav:21.19, Roll:12.78, Drag:03.80, Resist:37.78, Accel:-221.29, Speed:03.11, Work:-367.03, Power:-183.51
Dist:12.24, Grad:00.08, Grav:67.29, Roll:25.13, Drag:14.74, Resist:107.16, Accel:253.07, Speed:06.12, Work:720.46, Power:360.23
Dist:07.81, Grad:00.00, Grav:00.00, Roll:16.09, Drag:06.00, Resist:22.10, Accel:-186.07, Speed:03.91, Work:-327.94, Power:-163.97
Dist:12.74, Grad:00.00, Grav:00.00, Roll:26.23, Drag:15.95, Resist:42.18, Accel:206.82, Speed:06.37, Work:498.01, Power:249.00
Dist:12.03, Grad:00.07, Grav:43.83, Roll:24.71, Drag:14.22, Resist:82.77, Accel:-29.88, Speed:06.01, Work:1

In [ ]:
# Plot the fancy calculated power vs the measured power
# First plot the actual values (smoothed with a moving average)

#Placeholder to get rid of the smaller size graph
thinkplot.Plot([0],[0],label='Placeholder')    
thinkplot.Show()

win = 50

for activity,count in df2.ActivityId.value_counts().items():
    tempdf = df2[df2.ActivityId == activity]
    cumwatts = tempdf.Watts2.cumsum()*1000/tempdf.ElapasedDuration
    cumcalcwatts = tempdf.CalculatedWork.cumsum()*1000/tempdf.ElapasedDuration

    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,tempdf.CalculatedPower.rolling(window=win,min_periods=0).mean(),label='Calculated MA Power')
    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,tempdf.Watts2.rolling(window=win, min_periods=0).mean(),label='Measured MA Power')
    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,(tempdf.AltitudeMeters-400),label='Altitude')
    thinkplot.Config(title='Moving Average ('+str(win)+' periods): Calculated vs Measured Power '+tempdf.Time.min())
    thinkplot.Config (xlim=(0,20))
    thinkplot.Show()



In [ ]:
# Plot the fancy calculated power vs the measured power
# Then plot the cumulative values (smoothed with a moving average)


#Placeholder to get rid of the smaller size graph
thinkplot.Plot([0],[0],label='Placeholder')    
thinkplot.Show()


for activity,count in df2.ActivityId.value_counts().items():
    tempdf = df2[df2.ActivityId == activity]
    cumwatts = tempdf.Watts2.cumsum()*tempdf.Duration2/tempdf.ElapasedDuration
    cumcalcwatts = tempdf.CalculatedWork.cumsum()*1000/tempdf.ElapasedDuration

    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,cumcalcwatts,label='Calculated Average Power')
    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,cumwatts,label='Measured Average Power')
    thinkplot.Plot(tempdf.ElapasedDuration/1000/60,(tempdf.AltitudeMeters-400),label='Altitude')
    thinkplot.Config(title='Cum Average Calculated vs Measured Power '+tempdf.Time.min(),xlim=(0,50))
    thinkplot.Show()

In [6]:
df2.to_csv('/Users/hbuhrmann/PycharmProjects/CycleStats/PowerOutputExperiments/FinalFiles/test_last_7_rides.csv')

In [ ]:
# REMEMBER TO BRING DIRECTION OF TRAVEL AND WIND DIRECTION AND STRENGTH INTO THE CALCULATED POWER FORMULA

# REMEMBER TO BRING AIR DENSITY CALCULATION INTO THE CALCULATED POWER FORMULA

# THIS WON'T BE NECESSARY OR NEEDED FOR THE INDOOR CYCLING DATA, BUT IT WILL FOR OUTDOOR

# https://www.gribble.org/cycling/air_density.html


In [ ]:
# ALSO REMEMBER TO ADJUST POWER CALCULATION WITH MOMENTUM - 
# THIS SHOULD HAVE AN IMPACT ON THE POWER CALCULATIONS AT THE BOTTOM OF A HILL, MUCH LESS SO AT THE TOP OF A HILL
# CALCULATED THE KINETIC ENERGY AT THE END OF AIN INTERVAL - = 0.5*Weight*Velocity^2
# REGARD THAT AS AN "OPENING BALANCE" TO BE DEDUCTED AGAINST THE WORK DONE IN THE SUBSEQUENT INTERVAL 
# BUT YOU WILL HAVE TO ADD A DAMPING FACTOR OF SOME SORTS - THIS IS GETTING A BIT COMPLICATED
